# NLP Coursework


## 0. Set-up

### Main imports

In [1]:
!pip install simpletransformers
!pip install tensorboardx
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.

In [3]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch

from collections import Counter
from ast import literal_eval
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [4]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  False


In [5]:
if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

### Fetching the Don't Patronize Me! data manager module

In [6]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
module_url = f"https://github.com/Perez-AlmendrosC/dontpatronizeme/blob/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://github.com/Perez-AlmendrosC/dontpatronizeme/blob/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv


In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py


In [7]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [8]:
from dont_patronize_me import DontPatronizeMe
dpm = DontPatronizeMe('.', '.')
dpm.load_task1()
train_df=dpm.train_task1_df
train_df.head()

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0


# Load paragraph IDs

In [27]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

print(trids)
print(teids)

      par_id                  label
0       4341  [1, 0, 0, 1, 0, 0, 0]
1       4136  [0, 1, 0, 0, 0, 0, 0]
2      10352  [1, 0, 0, 0, 0, 1, 0]
3       8279  [0, 0, 0, 1, 0, 0, 0]
4       1164  [1, 0, 0, 1, 1, 1, 0]
...      ...                    ...
8370    8380  [0, 0, 0, 0, 0, 0, 0]
8371    8381  [0, 0, 0, 0, 0, 0, 0]
8372    8382  [0, 0, 0, 0, 0, 0, 0]
8373    8383  [0, 0, 0, 0, 0, 0, 0]
8374    8384  [0, 0, 0, 0, 0, 0, 0]

[8375 rows x 2 columns]
      par_id                  label
0       4046  [1, 0, 0, 1, 0, 0, 0]
1       1279  [0, 1, 0, 0, 0, 0, 0]
2       8330  [0, 0, 1, 0, 0, 0, 0]
3       4063  [1, 0, 0, 1, 1, 1, 0]
4       4089  [1, 0, 0, 0, 0, 0, 0]
...      ...                    ...
2089   10462  [0, 0, 0, 0, 0, 0, 0]
2090   10463  [0, 0, 0, 0, 0, 0, 0]
2091   10464  [0, 0, 0, 0, 0, 0, 0]
2092   10465  [0, 0, 0, 0, 0, 0, 0]
2093   10466  [0, 0, 0, 0, 0, 0, 0]

[2094 rows x 2 columns]


In [28]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

data=dpm.train_task1_df

data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


## Rebuild training set

In [30]:
from sklearn.model_selection import train_test_split
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

trdf1 = pd.DataFrame(rows)

In [36]:
# Split into internal train and internal dev
# 75-25 split so dev size matches test size
traindf1, devdf1 = train_test_split(trdf1, test_size=0.25, stratify=trdf1['label'], random_state=42)
traindf1

,par_id,community,text,label
6642,6477,refugee,Tensions remain high at Australia 's island re...,0
5187,4850,poor-families,"""Picariello , also known as """" Emperor Pic , ""...",0
3017,2473,refugee,"""Irungu , who also serves as the head of the N...",0
5638,5344,disabled,"Mrs. Aviva Dankner , owner of Castra Mall , al...",0
7183,7070,hopeless,2 : The ' Check Engine ' and ' ABS ' lights ca...,0
...,...,...,...,...
1401,679,in-need,"""Many argue this is inevitable -- that rising ...",0
8324,8333,refugee,"The mother of 6 , including 3 adopted children...",0
1199,449,disabled,HSE says it ca n't pay for services to help di...,0
2985,2437,in-need,A second T-Home project is being launched in t...,0


In [37]:
traindf1[['par_id']].to_csv('internal_train_par_ids.csv', index=False)
devdf1[['par_id']].to_csv('internal_dev_par_ids.csv', index=False)

In [15]:
# check that the split is stratified
print('Train label distribution:')
print(traindf1.label.value_counts(normalize=True))
print('Dev label distribution:')
print(devdf1.label.value_counts(normalize=True))


Train label distribution:
label
0    0.90527
1    0.09473
Name: proportion, dtype: float64
Dev label distribution:
label
0    0.904967
1    0.095033
Name: proportion, dtype: float64


## Load internal train and dev set

In [38]:
train_ids = pd.read_csv('internal_train_par_ids.csv')
dev_ids = pd.read_csv('internal_dev_par_ids.csv')
train_ids.par_id = train_ids.par_id.astype(str)
dev_ids.par_id = dev_ids.par_id.astype(str)

In [39]:
from sklearn.model_selection import train_test_split
rows = [] # will contain par_id, label and text
for idx in range(len(train_ids)):
  parid = train_ids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

traindf = pd.DataFrame(rows)

In [40]:
from sklearn.model_selection import train_test_split
rows = [] # will contain par_id, label and text
for idx in range(len(dev_ids)):
  parid = dev_ids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

devdf = pd.DataFrame(rows)

In [41]:
traindf

,par_id,community,text,label
0,6477,refugee,Tensions remain high at Australia 's island re...,0
1,4850,poor-families,"""Picariello , also known as """" Emperor Pic , ""...",0
2,2473,refugee,"""Irungu , who also serves as the head of the N...",0
3,5344,disabled,"Mrs. Aviva Dankner , owner of Castra Mall , al...",0
4,7070,hopeless,2 : The ' Check Engine ' and ' ABS ' lights ca...,0
...,...,...,...,...
6276,679,in-need,"""Many argue this is inevitable -- that rising ...",0
6277,8333,refugee,"The mother of 6 , including 3 adopted children...",0
6278,449,disabled,HSE says it ca n't pay for services to help di...,0
6279,2437,in-need,A second T-Home project is being launched in t...,0


In [42]:
devdf

,par_id,community,text,label
0,4928,hopeless,""""""" I went to Vauxhall High School , graduated...",0
1,3847,refugee,""""""" These children were predominantly Burmese ...",0
2,279,women,A submission from the Irish Women 's Council o...,0
3,1038,homeless,But displaced people were not left homeless by...,1
4,1879,immigrant,Studies reveal that the core of immigrant entr...,0
...,...,...,...,...
2089,10027,vulnerable,"Here in Jamaica , it is an unfortunate fact th...",1
2090,6857,hopeless,The hospital has managed to find some extra wa...,0
2091,5847,homeless,"Last year , a record 85 homes were demolished ...",0
2092,3189,in-need,Persons in need of a wheelchair or know of any...,0


## Rebuild test set

In [ ]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

import random
random.shuffle(rows)

testdf = pd.DataFrame(rows)
testdf

,par_id,community,text,label
0,8760,disabled,"""So , while making your workplace documentatio...",0
1,9423,in-need,Christmas is celebration of the birth of not m...,0
2,8677,homeless,REUTERSThe hands of a homeless man are seen as...,0
3,9936,hopeless,"Since its inception in 1994 , the NCVT has had...",0
4,187,in-need,""""""" We share a global responsibility to respon...",1
...,...,...,...,...
2089,8477,women,The Presidential Candidate of the New Party Pa...,0
2090,8799,women,The degree to which NSGD has helped improve th...,0
2091,10012,homeless,""""""" An estimated two million people have been ...",0
2092,9316,migrant,Thousands of people rallied in Melbourne Satur...,0
